In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/')

In [2]:
import torch
import pandas as pd
import numpy as np
import statistics
import torch
import random
import time
import numpy as np
#from transformers import AdamW
from torch.utils.tensorboard import SummaryWriter
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import shutil
import itertools
import os
from pathlib import Path
from model.GRU_model import *

In [3]:
from pipeline_config import *
from utils import *

In [4]:
# Seeds for comparisons:

torch.manual_seed(1)
np.random.seed(2)
random.seed(3)
#torch.use_deterministic_algorithms(True)

In [5]:
Lr

0.1

In [6]:
with open(train_dataset_path, 'rb') as f:
    train_dataset_dict = pickle.load(f)
    
with open(test_dataset_path, 'rb') as f:
    test_dataset_dict = pickle.load(f)

with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

with open(son_parent_path, 'rb') as f:
    son_parent_dict = pickle.load(f)

with open(parent_to_son_list_path, 'rb') as f:
    parent_to_son_list_dict = pickle.load(f)

with open(hgru_model_weights_path, 'rb') as f:
    hgru_weight_dict = pickle.load(f)

with open(sgru_model_weights_path, 'rb') as f:
    sgru_weight_dict = pickle.load(f)

with open(coefficient_dict_path, 'rb') as f:
    coefficient_dict = pickle.load(f)

In [7]:
cpi_us_dataset = pd.read_csv('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/resources/cpi_us_dataset.csv')
cpi_us_dataset

,Date,Category,Category_id,Price,Weight,Indent,Parent,Parent_ID
0,1984-01-15,All items,8106,101.900,100.000,0,NaN,NaN
1,1984-02-15,All items,8106,102.400,100.000,0,NaN,NaN
2,1984-03-15,All items,8106,102.600,100.000,0,NaN,NaN
3,1984-04-15,All items,8106,103.100,100.000,0,NaN,NaN
4,1984-05-15,All items,8106,103.400,100.000,0,NaN,NaN
...,...,...,...,...,...,...,...,...
53281,2019-03-15,Women's footwear,4044,122.974,0.293,5,Footwear,7182.0
53282,2019-03-15,Women's outerwear,7718,106.381,0.073,6,Women's apparel,8387.0
53283,2019-03-15,Women's suits and separates,9350,74.677,0.550,6,Women's apparel,8387.0
53284,2019-03-15,"Women's underwear, nightwear, sportswear and a...",9498,104.718,0.283,6,Women's apparel,8387.0


In [8]:
cpi_us_dataset[cpi_us_dataset['Parent_ID'] == 8830.0].Weight.count()

174

In [9]:
cpi_us_dataset[cpi_us_dataset['Parent'] == 'All items'].Weight.unique()

array([93.007     , 84.201     , 80.232     , 92.892     , 72.052     ,
        5.897     , 43.547     , 27.749     ,  6.993     ,  4.351     ,
       17.398     , 41.394     ,  7.108     , 32.977     , 17.178     ,
       11.845     ,  6.897     , 56.453     , 50.632     , 29.116     ,
       16.954     , 93.035     , 84.162     , 80.111     , 92.734     ,
       71.958     ,  5.656     , 43.386     , 27.548     ,  6.965     ,
        4.335     , 17.412     , 41.187     ,  7.266     , 32.788     ,
       16.95      , 11.852     ,  7.005     , 56.614     , 50.64      ,
       29.192     , 17.139     , 93.3       , 84.234     , 79.898     ,
       92.638     , 71.711     ,  5.516     , 42.916     , 27.15      ,
        6.7       ,  4.367     , 17.332     , 41.346     ,  7.362     ,
       32.407     , 16.641     , 11.674     ,  7.123     , 57.084     ,
       51.004     , 29.414     , 16.953     , 92.951     , 84.087     ,
       80.        , 92.654     , 71.806     ,  5.33      , 42.87

In [10]:
cpi_us_dataset.columns

Index(['Date', 'Category', 'Category_id', 'Price', 'Weight', 'Indent',
       'Parent', 'Parent_ID'],
      dtype='object')

In [11]:
def rmdir(dirc):
    dirc = Path(dirc)
    for itm in dirc.iterdir():
        if itm.is_dir():
            rmdir(itm)
            print("Deleting", itm, ".... successful.")
        else:
            itm.unlink()
    dirc.rmdir()

In [12]:
def bi_directional_model(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weights_path, hgru_weight_dict, coefficient_dict):
    # deleting files in models_weights folder
    path = os.getcwd()
    desired_path = path + '/models_weights/'
    is_empty = not any(Path(desired_path).iterdir())

    if is_empty==False:
        rmdir(Path(desired_path))
    
    # deleting empty folder
    try:
        os.rmdir(desired_path)
    except OSError:
        print ("Deletion of the directory %s failed" % desired_path)
    else:
        print ("Successfully deleted the directory %s" % desired_path)

    # creating new folder
    try:
        os.mkdir(desired_path)
    except OSError:
        print ("Creation of the directory %s failed" % desired_path)
    else:
        print ("Successfully created the directory %s" % desired_path)
    #-----------------------------------------------------------------------------------------------------------------------------------------------#

    num_categories = 0
    loss_coef_1= 3.59220247941877*(10**(-14))
    loss_coef_2= 1.3417253313109117*(10**(-10))


    for indent in sorted(list(categories_per_indent_dict.keys()),reverse=True):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')
            if category_name == 'Food at home': #for now...
                continue

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]: #or (category, son_parent_dict[category]) not in son_parent_corr_dict.keys():
                loss_coef_1 = 0

            if (category not in list(parent_to_son_list_dict.keys())) or (set(parent_to_son_list_dict[category]['sons']).issubset(set(categories_per_indent_dict[indent+1]))):
                loss_coef_2 = 0
            
            print('------------------------------------------------------------------')

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = Model
            optimizer = Optimizer
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            
            training_and_evaluation(model, indent, train_dataloader, test_dataloader, optimizer, category, hgru_weight_dict, coefficient_dict, son_parent_dict, parent_to_son_list_dict, category_id_to_name_dict, loss_coef_1, loss_coef_2, alpha, saving_param_path)


In [13]:
bidrectional_models = bi_directional_model(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath, hgru_weight_dict, coefficient_dict)

Deletion of the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/bidirectional/models_weights/ failed
Successfully created the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/bidirectional/models_weights/
num categories: 1
category id|name: 9540.0|Bacon and related products
------------------------------------------------------------------
num categories: 2
category id|name: 8302.0|Breakfast sausage and related products
------------------------------------------------------------------
num categories: 3
category id|name: 2135.0|Fresh and frozen chicken parts
------------------------------------------------------------------
num categories: 4
category id|name: 3415.0|Fresh whole chicken
------------------------------------------------------------------
num categories: 5
category id|name: 4850.0|Frozen fish and seafood
------------------------------------------------------------------
num categories: 6
category id|name: 9972.0|Ham,

Save prediction dict:

In [14]:
def get_results_on_test_set(weights_path, train_dataset_dict, test_dataset_dict, categories = None):
    predictions_dict = {}
    if categories is None:
        categories = list(test_dataset_dict.keys())
        
    for category in categories:
        if category == 'Food and beverages':
            continue
        print(category)
        train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category], test_dataset_dict[category])
        basic_model =GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
        basic_optimizer = torch.optim.AdamW(basic_model.parameters(), lr=Lr)
        ckp_path = weights_path+category+'.pt'
        model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        predictions = get_predictions_on_test_set(model, test_dataloader)
        predictions_dict[category] = predictions
    return predictions_dict

In [15]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])

In [16]:
predictions_dict = get_results_on_test_set(weightspath, train_dataset_dict, test_dataset_dict, categories = categories)

Admission to movies, theaters, and concerts
Bacon, breakfast sausage, and related products
Bread other than white
Canned fruits
Canned vegetables
Checking account and other bank services
Chicken
Cookies
Crackers, bread, and cracker products
Dried beans, peas, and lentils
Frankfurters
Fresh cakes and cupcakes
Fresh fish and seafood
Fresh sweetrolls, coffeecakes, doughnuts
Frozen and refrigerated bakery products, pies, tarts, turnovers
Frozen vegetables
Ham
Intracity mass transit
Lunchmeats
Oranges, including tangerines
Other condiments
Other pork including roasts and picnics
Other uncooked poultry including turkey
Parking fees and tolls
Pork chops
Prepared salads
Processed fish and seafood
Salt and other seasonings and spices
Sauces and gravies
Ship fare
Uncooked beef roasts
Uncooked beef steaks
Uncooked ground beef
Uncooked other beef and veal
Video discs and other media
White bread
Admissions
Airline fare
Apparel services other than laundry and dry cleaning
Apples
Bananas
Beef and vea

In [17]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/bidirectional/test_predictions.pickle', 'wb') as handle:
    pickle.dump(predictions_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)